# Testing cnn for classifying universes
Nov 10, 2020


In [2]:
import argparse
import os
import random


import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML


import time
from datetime import datetime
import glob
import pickle
import yaml
import logging

In [3]:
%matplotlib widget

## Modules

In [4]:
def f_load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    return config

### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)



In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

# Generator Code
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

class Discriminator(nn.Module):
    def __init__(self, ngpu, nz,nc,ndf,n_classes,kernel_size,stride,d_padding):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            # nn.Conv2d(in_channels, out_channels, kernel_size,stride,padding,output_padding,groups,bias, Dilation,padding_mode)
            nn.Conv2d(nc, ndf,kernel_size, stride, d_padding,  bias=True),
            nn.BatchNorm2d(ndf,eps=1e-05, momentum=0.9, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, kernel_size, stride, d_padding, bias=True),
            nn.BatchNorm2d(ndf * 2,eps=1e-05, momentum=0.9, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size, stride, d_padding, bias=True),
            nn.BatchNorm2d(ndf * 4,eps=1e-05, momentum=0.9, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size, stride, d_padding, bias=True),
            nn.BatchNorm2d(ndf * 8,eps=1e-05, momentum=0.9, affine=True),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Flatten(),
            nn.Linear(nc*ndf*8*8*8, n_classes)
#             nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)



## Main code

In [6]:
torch.backends.cudnn.benchmark=True
t0=time.time()
#################################
###### Initialize variables #######
config_file='config_128.yaml'
config_dict=f_load_config(config_file)
print(config_dict)

workers=config_dict['training']['workers']
nc,nz,ngf,ndf=config_dict['training']['nc'],config_dict['training']['nz'],config_dict['training']['ngf'],config_dict['training']['ndf']
lr,beta1=config_dict['training']['lr'],config_dict['training']['beta1']
kernel_size,stride=config_dict['training']['kernel_size'],config_dict['training']['stride']
g_padding,d_padding=config_dict['training']['g_padding'],config_dict['training']['d_padding']
flip_prob=config_dict['training']['flip_prob']

image_size=config_dict['data']['image_size']
checkpoint_size=config_dict['data']['checkpoint_size']
num_imgs=config_dict['data']['num_imgs']
ip_fname=config_dict['data']['ip_fname']
op_loc=config_dict['data']['op_loc']

# Overriding configs in .yaml file (different for jupyter notebook)
ngpu=1
batch_size=128
spec_loss_flag=True
checkpoint_size=50
num_imgs=2000 # Number of images to use 
num_epochs=4
lr=0.0002
n_classes=6

### Initialize random seed (different for Jpt notebook)
manualSeed=21245
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
device = torch.device("cuda" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print('Device:',device)

#     #################################
#     ####### Read data and precompute ######
#     # ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
#     ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/data_x.npy'
#     labels='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/data_y.npy'

#     img=np.load(ip_fname)[:num_imgs].transpose(0,1,2,3)
#     t_img=torch.from_numpy(img)
#     print(img.shape,t_img.shape)

#     dataset=TensorDataset(t_img)
#     dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=1,drop_last=True)

#################################
###### Build Networks ###
print("Building CNN")
# Create Discriminator
netD = Discriminator(ngpu, nz,nc,ndf,n_classes,kernel_size,stride,g_padding).to(device)
netD.apply(weights_init)
print(netD)
summary(netD,(1,128,128))
# Handle multi-gpu if desired
ngpu=torch.cuda.device_count()

print("Number of GPUs used",ngpu)
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Initialize BCELoss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(netD.parameters(), lr=0.001, momentum=0.9)

#     fixed_noise = torch.randn(batch_size, 1, 1, nz, device=device) #Latent vectors to view G progress

# Setup Adam optimizers for both G and D
#     optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999),eps=1e-7)

#################################    
###### Set up directories ####### (different for Jpt notebook)
#     run_suffix='_nb_test'
#     ### Create prefix for foldername 
#     now=datetime.now()
#     fldr_name=now.strftime('%Y%m%d_%H%M%S') ## time format
#     # print(fldr_name)
#     save_dir=op_loc+fldr_name+run_suffix

#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir+'/models')
#         os.makedirs(save_dir+'/images')

# Fresh start    
#     iters = 0; start_epoch=0
#     best_chi1,best_chi2=1e10,1e10

{'description': 'GAN', 'data': {'ip_fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy', 'op_loc': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/', 'image_size': 128, 'checkpoint_size': 1, 'num_imgs': 200000}, 'training': {'workers': 2, 'nc': 1, 'nz': 64, 'ngf': 64, 'ndf': 64, 'lr': 0.0002, 'beta1': 0.5, 'kernel_size': 5, 'stride': 2, 'g_padding': 2, 'd_padding': 2, 'flip_prob': 0.01}}
Random Seed:  21245
Device: cuda
Building CNN
Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slo

In [7]:
# ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/data_x.npy'
# labels_file='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/data_y.npy'
# ids_file='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/data_id.npy'

# img=np.load(ip_fname)
# labels=np.load(labels_file)
# ids=np.load(ids_file)

# t_img=torch.from_numpy(img)
# print(img.shape,t_img.shape)

In [8]:
## Read data from dataframe
data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_4_four_universes_6k_cnn/'
df_data=pd.read_pickle(data_dir+'/df_data.pkle')
df_data=df_data.sample(frac=1,random_state=20).reset_index(drop=True)
train_size,val_size,test_size=0.7,0.1,0.1
data_size=df_data.shape[0]

In [9]:
df_data[['ID','label']].head()

,ID,label
0,30513,4
1,27797,4
2,4772,0
3,3405,0
4,13219,2


In [10]:
idx1,idx2,idx3=int(train_size*data_size),int((train_size+val_size)*data_size),int((train_size+val_size+test_size)*data_size)
print(idx1,idx2,idx3)

df_temp=df_data.loc[np.arange(0,idx1)]
dataset=TensorDataset(torch.Tensor(np.stack(df_temp.img.values)),torch.Tensor(df_temp.label.values))
train_loader=DataLoader(dataset,batch_size=batch_size,shuffle=True,num_workers=1,drop_last=True)

df_temp=df_data.loc[np.arange(idx1,idx2)]
dataset=TensorDataset(torch.Tensor(np.stack(df_temp.img.values)),torch.Tensor(df_temp.label.values))
val_loader=DataLoader(dataset,batch_size=16,shuffle=True,num_workers=1,drop_last=True)

df_temp=df_data.loc[np.arange(idx2,idx3)]
dataset=TensorDataset(torch.Tensor(np.stack(df_temp.img.values)),torch.Tensor(df_temp.label.values))
test_loader=DataLoader(dataset,batch_size=8,shuffle=True,num_workers=1,drop_last=True)



25804 29491 33177


In [11]:
## Test model
def f_test(data_loader,netD):
    netD.eval()
    correct,total=0,0
    with torch.no_grad():
        for count,data in enumerate(data_loader):
            images,labels=data[0].to(device),data[1].to(device)
            outputs=netD(images)
            _,predictions=torch.max(outputs,1)
            total+=labels.size(0)
            correct+=(predictions==labels).sum().item()

    accuracy=(correct/total)*100
#     print("Accuracy %",accuracy)
#     print(correct,total)
    return accuracy


In [15]:
accuracy=[]
for epoch in range(0,4):
    running_loss=0.0
    print("Epoch",epoch)
    for i, data in enumerate(train_loader):
    #     print(images.shape,labels.shape)
        images,labels=data[0].to(device),data[1].to(device)
        optimizer.zero_grad()

    #     netD.train();  ### Need to add these after inference and before training
        netD.zero_grad()
        labels=labels.long()
        output = netD(images)

        loss= criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        
        if i%10==0: accuracy.append(f_test(val_loader,netD))
        netD.train()


Epoch 0
Epoch 1
Epoch 2
Epoch 3


In [16]:
plt.figure()
plt.plot(accuracy)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
## Test model
f_test(test_loader,netD)

90.1086956521739

84